In [2]:
import pandas as pd
import openpyxl as px
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# データの読み込み
data = pd.read_excel('data.xlsx')

In [ ]:
# 概要を分割し、最初の3つを取得
data[['概要1', '概要2', '概要3']] = data['概要'].str.split('，', expand=True).iloc[:, :3]
data['概要1'] = data['概要1'].str.replace('（.*', '', regex=True)
data['概要2'] = data['概要2'].str.replace('（.*', '', regex=True)
data['概要3'] = data['概要3'].str.replace('（.*', '', regex=True)
data

In [ ]:
# ラベルエンコーディング
# 種類と概要の全てのユニークな文字列に連番を付与
type_columns = ['種類1', '種類2', '種類3']
summary_columns = ['概要1', '概要2', '概要3']
main_columns = ['主成分']
class_columns = ['分類']

# 連番リストとマッピングの作成
def create_mapping(data, columns):
    unique_values = pd.Series(pd.concat([data[col] for col in columns]).unique()).dropna()
    return pd.Series(index=unique_values, data=range(1, len(unique_values) + 1))
type_mapping = create_mapping(data, type_columns)
summary_mapping = create_mapping(data, summary_columns)
main_mapping = create_mapping(data, main_columns)
class_mapping = create_mapping(data, class_columns)

# Excel出力
with pd.ExcelWriter('mappings.xlsx') as writer:
    type_mapping.to_excel(writer, sheet_name='種類連番')
    summary_mapping.to_excel(writer, sheet_name='概要連番')
    main_mapping.to_excel(writer, sheet_name='主成分連番')
    class_mapping.to_excel(writer, sheet_name='分類連番')

# マッピングの適用
for mapping in [type_mapping, summary_mapping, main_mapping, class_mapping]:
    for col in type_columns:
        data[col] = data[col].map(mapping)

In [ ]:
# 0埋め
data.fillna(0, inplace=True)